# Data Cleaning

In [123]:
# import libraries
import pandas as pd
import numpy as np
import ast

# sklearn libraries
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Justwatch API
from justwatch import JustWatch

In [2]:
# read in dataset
df = pd.read_csv('./data/titles.csv', index_col=0)

In [3]:
df.head(3)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
0,ts20740,20740,Dragon Ball Z,/poster/8569195/{profile},Dragon Ball Z is a Japanese animated televisio...,1989.0,show,805.0,387.966,8.8,128409.0,8.286,tt0121220,26447.0,"[14, 1, 2, 3, 7, 12, 6]",TV-PG,24.0,['JP'],16,Funimation
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013.0,show,55.0,89.689,9.0,325381.0,8.643,tt2560140,205148.0,"[1, 14, 2, 6, 7, 9]",TV-MA,24.0,['JP'],4,Funimation
2,ts21560,21560,Dragon Ball,/poster/290552685/{profile},"Long ago in the mountains, a fighting master k...",1986.0,show,1936.0,15.964,8.6,56606.0,8.218,tt0088509,210469.0,"[2, 1, 3, 7, 14, 12]",TV-14,24.0,['JP'],10,Funimation


In [4]:
# info of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1941 entries, 0 to 0
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   jw_entity_id          1941 non-null   object 
 1   id                    1941 non-null   int64  
 2   title                 1941 non-null   object 
 3   poster                1930 non-null   object 
 4   description           1908 non-null   object 
 5   release_year          1940 non-null   float64
 6   type                  1941 non-null   object 
 7   imdb_popularity       369 non-null    float64
 8   tmdb_popularity       1924 non-null   float64
 9   imdb_score            1699 non-null   float64
 10  imdb_votes            1699 non-null   float64
 11  tmdb_score            1854 non-null   float64
 12  imdb_id               1722 non-null   object 
 13  tmdb_id               1924 non-null   float64
 14  genre_ids             1931 non-null   object 
 15  age_certification     15

In [5]:
df.isna().sum()

jw_entity_id               0
id                         0
title                      0
poster                    11
description               33
release_year               1
type                       0
imdb_popularity         1572
tmdb_popularity           17
imdb_score               242
imdb_votes               242
tmdb_score                87
imdb_id                  219
tmdb_id                   17
genre_ids                 10
age_certification        432
runtime                    1
production_countries      26
seasons                    0
streaming_app              0
dtype: int64

In [6]:
df.duplicated().sum()

1

In [7]:
df.loc[df.duplicated(keep=False)]

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
89,tm102863,102863,Fafner in the Azure: Dead Aggressor - Heaven a...,/poster/174761052/{profile},The year is 2148. Two years have passed since...,2010.0,movie,NaN,5.287,6.4,70.0,3.8,tt1794963,137502.0,"[1, 2, 6, 14]",NaN,95.0,['JP'],0,Funimation
90,tm102863,102863,Fafner in the Azure: Dead Aggressor - Heaven a...,/poster/174761052/{profile},The year is 2148. Two years have passed since...,2010.0,movie,NaN,5.287,6.4,70.0,3.8,tt1794963,137502.0,"[1, 2, 6, 14]",NaN,95.0,['JP'],0,Funimation


The `jw_entity_id` column should be the unique identifier for each movie so if there are duplicates of this then there are duplicate movies.

In [8]:
# duplicates for jw_entity_id
df.jw_entity_id.duplicated().sum()

345

In [9]:
df.drop_duplicates(subset=['jw_entity_id']).title.duplicated().sum()

3

In [10]:
df.drop_duplicates(subset=['jw_entity_id']).loc[df.drop_duplicates(subset=['jw_entity_id']).title.duplicated(keep=False)].sort_values('jw_entity_id')

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
41,tm299419,299419,Tokyo Ghoul,/poster/30444162/{profile},A Tokyo college student is attacked by a ghoul...,2017.0,movie,NaN,48.513,5.7,3649.0,7.000,tt5815944,433945.0,"[12, 1, 6, 14, 7, 9]",NC-17,119.0,['JP'],0,Funimation
41,ts20202,20202,Tokyo Ghoul,/poster/249116603/{profile},Ken Kaneki is a bookworm college student who m...,2014.0,show,700.0,155.001,7.8,52338.0,8.475,tt3741634,61374.0,"[1, 7, 9, 12, 2, 6]",TV-MA,24.0,['JP'],4,Funimation
552,ts272319,272319,The Duke of Death and His Maid,/poster/247591590/{profile},"Due to a childhood curse, anything that the Du...",2021.0,show,NaN,32.898,NaN,NaN,8.094,NaN,117992.0,"[2, 3, 6]",TV-14,24.0,['JP'],1,Funimation
611,ts280992,280992,The Duke of Death and His Maid,/poster/249239431/{profile},A cursed duke who kills everyone he touches li...,2021.0,show,NaN,NaN,7.2,552.0,NaN,tt13971512,NaN,"[2, 3, 6, 13]",TV-14,23.0,NaN,2,Crunchyroll
166,ts31181,31181,Fruits Basket,/poster/155050012/{profile},Tohru Honda is 16 year old orphaned girl who g...,2001.0,show,3548.0,15.592,7.9,5822.0,7.364,tt0328738,36941.0,"[6, 2, 3, 7, 13]",TV-PG,24.0,['JP'],1,Funimation
52,ts87522,87522,Fruits Basket,/poster/246787476/{profile},After a family tragedy turns her life upside d...,2019.0,show,769.0,43.038,8.6,6258.0,8.306,tt9304350,85991.0,"[2, 3, 6, 7, 13]",TV-14,24.0,['JP'],3,Funimation


In [11]:
df.loc[df['jw_entity_id'].duplicated(keep=False)].sort_values('jw_entity_id')

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
89,tm102863,102863,Fafner in the Azure: Dead Aggressor - Heaven a...,/poster/174761052/{profile},The year is 2148. Two years have passed since...,2010.0,movie,NaN,5.287000,6.4,70.0,3.800,tt1794963,137502.0,"[1, 2, 6, 14]",NaN,95.0,['JP'],0,Funimation
90,tm102863,102863,Fafner in the Azure: Dead Aggressor - Heaven a...,/poster/174761052/{profile},The year is 2148. Two years have passed since...,2010.0,movie,NaN,5.287000,6.4,70.0,3.800,tt1794963,137502.0,"[1, 2, 6, 14]",NaN,95.0,['JP'],0,Funimation
304,ts102805,102805,Fate/Grand Order - Absolute Demonic Front: Bab...,/poster/219416423/{profile},Chaldea agents Ritsuka Fujimnaru and Mash Kyri...,2019.0,show,4048.0,1.099979,7.3,749.0,8.006,tt9525238,76143.0,"[2, 1, 6, 7, 8, 14]",NaN,23.0,['JP'],1,Funimation
347,ts102805,102805,Fate/Grand Order - Absolute Demonic Front: Bab...,/poster/219416423/{profile},Chaldea agents Ritsuka Fujimnaru and Mash Kyri...,2019.0,show,4048.0,1.099979,7.3,749.0,8.006,tt9525238,76143.0,"[2, 1, 6, 7, 8, 14]",NaN,23.0,['JP'],1,Crunchyroll
79,ts11151,11151,Fate/Zero,/poster/253936288/{profile},War of the Holy Grail - Pursuing the power of ...,2011.0,show,1877.0,44.946000,8.2,13048.0,8.100,tt2051178,45845.0,"[14, 1, 2, 6, 7, 12]",TV-MA,24.0,['JP'],2,Crunchyroll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,ts90371,90371,Mobile Suit Gundam: The Origin - Advent of the...,/poster/144808814/{profile},"This is the story of how Char Aznable, the Red...",2019.0,show,NaN,4.035000,7.9,72.0,8.000,tt12899130,88865.0,"[2, 1, 14, 16, 6]",TV-14,25.0,['JP'],1,Crunchyroll
156,ts9246,9246,Blue Exorcist,/poster/255362748/{profile},"Humans live in the world of Assiah, demons in ...",2011.0,show,2879.0,44.129000,7.4,13187.0,8.049,tt1799631,38464.0,"[14, 1, 6, 7, 9, 10, 2]",TV-14,24.0,['JP'],2,Crunchyroll
148,ts9246,9246,Blue Exorcist,/poster/255362748/{profile},"Humans live in the world of Assiah, demons in ...",2011.0,show,2879.0,44.129000,7.4,13187.0,8.049,tt1799631,38464.0,"[14, 1, 6, 7, 9, 10, 2]",TV-14,24.0,['JP'],2,Funimation
683,ts9686,9686,Arakawa Under the Bridge,/poster/300679177/{profile},"Ko Ichinomiya's family motto is ""Never be inde...",2010.0,show,NaN,5.075000,7.1,582.0,6.600,tt1639471,92354.0,"[2, 3, 7, 13]",TV-14,30.0,['JP'],2,Crunchyroll


### Drop duplicate rows

In [12]:
# drop duplicates by jw_entity_id
df.drop_duplicates(subset=['jw_entity_id'], keep='last', inplace=True)

In [13]:
df.loc[df['title'] == 'Attack on Titan']

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
1,ts20682,20682,Attack on Titan,/poster/174708726/{profile},"Several hundred years ago, humans were nearly ...",2013.0,show,55.0,89.689,9.0,325381.0,8.643,tt2560140,205148.0,"[1, 14, 2, 6, 7, 9]",TV-MA,24.0,['JP'],4,Crunchyroll


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1596 entries, 17 to 0
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   jw_entity_id          1596 non-null   object 
 1   id                    1596 non-null   int64  
 2   title                 1596 non-null   object 
 3   poster                1585 non-null   object 
 4   description           1563 non-null   object 
 5   release_year          1595 non-null   float64
 6   type                  1596 non-null   object 
 7   imdb_popularity       249 non-null    float64
 8   tmdb_popularity       1580 non-null   float64
 9   imdb_score            1383 non-null   float64
 10  imdb_votes            1383 non-null   float64
 11  tmdb_score            1515 non-null   float64
 12  imdb_id               1405 non-null   object 
 13  tmdb_id               1580 non-null   float64
 14  genre_ids             1586 non-null   object 
 15  age_certification     1

In [15]:
df.isna().sum()

jw_entity_id               0
id                         0
title                      0
poster                    11
description               33
release_year               1
type                       0
imdb_popularity         1347
tmdb_popularity           16
imdb_score               213
imdb_votes               213
tmdb_score                81
imdb_id                  191
tmdb_id                   16
genre_ids                 10
age_certification        411
runtime                    1
production_countries      25
seasons                    0
streaming_app              0
dtype: int64

To develop a Minumum Viable Product(MVP) I will be focusing on a few of the features from this dataset, and if there is time I will focus on handling null values to hopefully improve upon the recommender system.

I'll use the following columns to create an MVP: `title`, `release_year`, `type`, `genre_ids`, `runtime`, and `seasons`

I'll focus on handling the null values in these columns.

## Handling Null Values

#### `release_year`

In [16]:
# locating release_year null values
df.loc[df['release_year'].isna()]

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
758,ts61912,61912,Japanese Anime Classic Collection,NaN,NaN,NaN,show,NaN,0.6,NaN,NaN,NaN,NaN,56020.0,NaN,NaN,7.0,NaN,1,Crunchyroll


I researched online and found on Amazon that this collection was released in 2007. I will replace it's `release_year` value with 2007

In [17]:
# replacing value with 2007
df['release_year'].fillna(2007, inplace=True)

In [18]:
# check if it worked
df.loc[df['title'] == 'Japanese Anime Classic Collection']

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
758,ts61912,61912,Japanese Anime Classic Collection,NaN,NaN,2007.0,show,NaN,0.6,NaN,NaN,NaN,NaN,56020.0,NaN,NaN,7.0,NaN,1,Crunchyroll


#### `genre_ids`

In [19]:
# get list of genres from justwatch
just_watch = JustWatch(country='US')

In [20]:
# genre list of dictionaries
genre_list = just_watch.get_genres()

In [21]:
genre_list

[{'id': 1,
  'short_name': 'act',
  'technical_name': 'action',
  'translation': 'Action & Adventure',
  'slug': 'action-and-adventure'},
 {'id': 2,
  'short_name': 'ani',
  'technical_name': 'animation',
  'translation': 'Animation',
  'slug': 'animation'},
 {'id': 3,
  'short_name': 'cmy',
  'technical_name': 'comedy',
  'translation': 'Comedy',
  'slug': 'comedy'},
 {'id': 4,
  'short_name': 'crm',
  'technical_name': 'crime',
  'translation': 'Crime',
  'slug': 'crime'},
 {'id': 5,
  'short_name': 'doc',
  'technical_name': 'documentation',
  'translation': 'Documentary',
  'slug': 'documentary'},
 {'id': 6,
  'short_name': 'drm',
  'technical_name': 'drama',
  'translation': 'Drama',
  'slug': 'drama'},
 {'id': 7,
  'short_name': 'fnt',
  'technical_name': 'fantasy',
  'translation': 'Fantasy',
  'slug': 'fantasy'},
 {'id': 8,
  'short_name': 'hst',
  'technical_name': 'history',
  'translation': 'History',
  'slug': 'history'},
 {'id': 9,
  'short_name': 'hrr',
  'technical_name'

In [22]:
# genre dictionary
genre_dict = {}

# for loop to get info from genre_list
for genre in genre_list:
    genre_dict[genre['id']] = genre['translation']

In [23]:
genre_dict

{1: 'Action & Adventure',
 2: 'Animation',
 3: 'Comedy',
 4: 'Crime',
 5: 'Documentary',
 6: 'Drama',
 7: 'Fantasy',
 8: 'History',
 9: 'Horror',
 10: 'Kids & Family',
 11: 'Music & Musical',
 12: 'Mystery & Thriller',
 13: 'Romance',
 14: 'Science-Fiction',
 15: 'Sport',
 16: 'War & Military',
 17: 'Western',
 23: 'Reality TV',
 18: 'Made in Europe'}

In [24]:
# locate missing genre_ids
df.loc[df['genre_ids'].isna()]

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,imdb_votes,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app
730,ts252020,252020,Scared Rider Xechs,/poster/253937855/{profile},"The Blue World, which symbolizes reason, is un...",2016.0,show,NaN,1.187000,NaN,NaN,7.0,tt13774780,107208.0,NaN,NaN,24.0,['JP'],1,Funimation
737,ts27040,27040,Gunslinger Girl -Il Teatrino-,NaN,NaN,2008.0,show,NaN,0.217056,NaN,NaN,5.0,NaN,26605.0,NaN,NaN,25.0,NaN,1,Funimation
477,ts297686,297686,Crunchyroll Anime Awards,/poster/247410363/{profile},Annual awards ceremony by the anime streaming ...,2017.0,show,NaN,2.853000,NaN,NaN,NaN,NaN,128427.0,NaN,NaN,90.0,['US'],5,Crunchyroll
758,ts61912,61912,Japanese Anime Classic Collection,NaN,NaN,2007.0,show,NaN,0.600000,NaN,NaN,NaN,NaN,56020.0,NaN,NaN,7.0,NaN,1,Crunchyroll
818,ts215898,215898,Hakata Mentai! Pirikarako-chan,/poster/136330187/{profile},Hakata Mentai! Pirikarako-chan is set in a mys...,2019.0,show,NaN,1.313000,NaN,NaN,5.3,NaN,90847.0,NaN,NaN,4.0,['JP'],1,Crunchyroll
972,ts94758,94758,The Journey Home,/poster/146688947/{profile},Insects are taken up into space for use in exp...,2015.0,show,NaN,NaN,7.0,15.0,NaN,tt6667152,NaN,NaN,TV-G,23.0,['CA'],20,Crunchyroll
983,ts84363,84363,Asenshu Anime Recap,/poster/85505445/{profile},"Anime Synopsis, News and Spoilers in Albanian.",2018.0,show,NaN,0.608000,NaN,NaN,NaN,NaN,82824.0,NaN,NaN,3.0,['AL'],1,Crunchyroll
996,ts208160,208160,"Demian, o Justiceiro",/poster/251015786/{profile},NaN,1968.0,show,NaN,NaN,NaN,NaN,NaN,tt0243695,NaN,NaN,NaN,19.0,NaN,1,Crunchyroll
1032,ts341696,341696,Sony Music AnimeSongs ONLINE 2022,NaN,"Sony Music AnimeSongs ONLINE 2022"" is a festiv...",2022.0,show,NaN,0.600000,NaN,NaN,10.0,NaN,194597.0,NaN,NaN,208.0,['JP'],1,Crunchyroll
1037,ts53246,53246,Fan Service,NaN,"Gray Haddock, Kerry Shawcross, Miles Luna, and...",2016.0,show,NaN,1.498000,NaN,NaN,NaN,NaN,69193.0,NaN,NaN,75.0,['US'],4,Crunchyroll


There are 10 rows that don't have genres attached to them. I will have to use an outside source(website) to find genres for them. If I am unable to find a genre for them I will list it as 'other'.

In [25]:
missing_genres = list(df.loc[df['genre_ids'].isna()].title)
missing_genres

['Scared Rider Xechs',
 'Gunslinger Girl -Il Teatrino-',
 'Crunchyroll Anime Awards',
 'Japanese Anime Classic Collection',
 'Hakata Mentai! Pirikarako-chan',
 'The Journey Home',
 'Asenshu Anime Recap',
 'Demian, o Justiceiro',
 'Sony Music AnimeSongs ONLINE 2022',
 'Fan Service']

In [26]:
# filling the values of the missing genres
missing_genres_dict = {
    'Scared Rider Xechs': ['Action & Adventure', 'Romance', 'Science-Fiction'],
    'Gunslinger Girl -Il Teatrino-': ['Science-Fiction', 'Action & Adventure'],
    'Crunchyroll Anime Awards': ['Other'],
    'Japanese Anime Classic Collection': ['Other'],
    'Hakata Mentai! Pirikarako-chan': ['Comedy'],
    'The Journey Home': ['Action & Adventure', 'Comedy', 'Kids & Family'],
    'Asenshu Anime Recap': ['Other'],
    'Demian, o Justiceiro': ['Science-Fiction', 'Action & Adventure'],
    'Sony Music AnimeSongs ONLINE 2022': ['Music & Musical'],
    'Fan Service': ['Other'] 
}

In [27]:
# function to take in title and return list of genres
def return_genre(title, genre_dict):
    return genre_dict[title]

for title in missing_genres:
    print(return_genre(title, missing_genres_dict))

['Action & Adventure', 'Romance', 'Science-Fiction']
['Science-Fiction', 'Action & Adventure']
['Other']
['Other']
['Comedy']
['Action & Adventure', 'Comedy', 'Kids & Family']
['Other']
['Science-Fiction', 'Action & Adventure']
['Music & Musical']
['Other']


In [28]:
df2 = df.apply(lambda x: return_genre(x['title'], missing_genres_dict) if pd.isnull(x['genre_ids']) else x['genre_ids'], axis=1)

In [29]:
df2

17                     [14, 2, 1]
23            [1, 2, 6, 4, 12, 3]
25              [14, 12, 2, 6, 9]
30      [14, 2, 1, 16, 17, 4, 12]
31              [14, 6, 9, 12, 2]
                  ...            
1088                [2, 12, 7, 9]
1089                    [2, 3, 6]
1090                       [2, 3]
1091                       [2, 3]
0                    [1, 2, 6, 7]
Length: 1596, dtype: object

In [30]:
df['genre_ids_fill'] = df2

In [31]:
df

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,...,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app,genre_ids_fill
17,ts28221,28221,Robotech,/poster/260247952/{profile},Robotech is an 85-episode adaptation of three ...,1985.0,show,4459.0,26.408,8.5,...,8.600,tt3133870,2192.0,"[14, 2, 1]",NaN,24.0,['US'],3,Funimation,"[14, 2, 1]"
23,ts199,199,Black Lagoon,/poster/300561407/{profile},The story follows a team of pirate mercenaries...,2006.0,show,1835.0,30.388,7.9,...,8.071,tt0962826,12598.0,"[1, 2, 6, 4, 12, 3]",TV-MA,25.0,['JP'],3,Funimation,"[1, 2, 6, 4, 12, 3]"
25,ts25674,25674,Serial Experiments Lain,/poster/177246788/{profile},Lain—driven by the abrupt suicide of a classma...,1998.0,show,3066.0,20.537,8.1,...,8.300,tt0500092,1087.0,"[14, 12, 2, 6, 9]",TV-MA,25.0,['JP'],1,Funimation,"[14, 12, 2, 6, 9]"
30,ts20388,20388,Psycho-Pass,/poster/139158745/{profile},Psycho-Pass is set in a futuristic era in Japa...,2012.0,show,1891.0,51.148,8.2,...,7.824,tt2379308,94878.0,"[14, 2, 1, 16, 17, 4, 12]",TV-MA,27.0,['JP'],4,Funimation,"[14, 2, 1, 16, 17, 4, 12]"
31,ts27309,27309,Paranoia Agent,/poster/245190109/{profile},An elementary schooler repeatedly attacks peop...,2004.0,show,3535.0,16.523,8.0,...,8.112,tt0433722,136272.0,"[14, 6, 9, 12, 2]",TV-MA,24.0,['JP'],1,Funimation,"[14, 6, 9, 12, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,ts249933,249933,Ninja Collection,/poster/227023124/{profile},Yamishibai: Japanese Ghost Stories spin-off. T...,2020.0,show,NaN,4.069,5.2,...,6.000,tt12706364,105045.0,"[2, 12, 7, 9]",NaN,4.0,['JP'],1,Crunchyroll,"[2, 12, 7, 9]"
1089,ts77748,77748,Slow Start,/poster/38897015/{profile},Hana Ichinose is a normal 16-year-old high sch...,2018.0,show,NaN,5.246,6.1,...,7.000,tt7899036,76099.0,"[2, 3, 6]",NaN,23.0,['JP'],1,Crunchyroll,"[2, 3, 6]"
1090,ts134409,134409,Magical Somera-chan,/poster/246627980/{profile},The story follows the everyday life of Somera ...,2015.0,show,NaN,0.942,5.4,...,2.000,tt5502130,90327.0,"[2, 3]",NaN,3.0,['JP'],1,Crunchyroll,"[2, 3]"
1091,ts43287,43287,JK-Meshi!,/poster/246628102/{profile},Three high school girls have mastered the art ...,2015.0,show,NaN,1.307,6.6,...,2.300,tt5513012,67049.0,"[2, 3]",NaN,3.0,['JP'],1,Crunchyroll,"[2, 3]"


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1596 entries, 17 to 0
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   jw_entity_id          1596 non-null   object 
 1   id                    1596 non-null   int64  
 2   title                 1596 non-null   object 
 3   poster                1585 non-null   object 
 4   description           1563 non-null   object 
 5   release_year          1596 non-null   float64
 6   type                  1596 non-null   object 
 7   imdb_popularity       249 non-null    float64
 8   tmdb_popularity       1580 non-null   float64
 9   imdb_score            1383 non-null   float64
 10  imdb_votes            1383 non-null   float64
 11  tmdb_score            1515 non-null   float64
 12  imdb_id               1405 non-null   object 
 13  tmdb_id               1580 non-null   float64
 14  genre_ids             1586 non-null   object 
 15  age_certification     1

Now I need to transform the `genre_ids_fill` from numbers in the list to the genres they correspond to. I'll create a new function that will take in a list and returns a new list of the genres corresponding to the numbers.

First I'll have to change the values of the genre_ids_fill column because the lists are stored as strings and not lists.

In [33]:
# change strings of list to lists
df['genre_ids_fill'] = df['genre_ids_fill'].map(lambda x: ast.literal_eval(x) if type(x) == str else x)

In [34]:
def num_to_genre(num_list, genre_dict):
    # create empty list for genres
    genre_list = []
    
    for num in num_list:
        
        # append genre to new list
        genre_list.append(genre_dict[num])
        
    return genre_list

In [35]:
df2 = df.apply(lambda x: num_to_genre(x['genre_ids_fill'], genre_dict) 
               if type(x['genre_ids_fill'][0]) == int else x['genre_ids_fill'], 
               axis=1)

In [36]:
df['genre_ids_fill'] = df2

I successfully transformed the genre_ids from numbers to the strings of the genres they represent. 

Next, I will fill the null value in the `runtime` column.

#### `runtime`

I have to locate where the null value is first. I also have to reset the index so it starts at 0.

In [37]:
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [38]:
# locating the row with the missing value
df.loc[df['runtime'].isna()]

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,...,tmdb_score,imdb_id,tmdb_id,genre_ids,age_certification,runtime,production_countries,seasons,streaming_app,genre_ids_fill
1234,ts26242,26242,Kite Liberator,/poster/272395473/{profile},"KITE Liberator is an American-released, Japane...",2008.0,show,NaN,2.346,NaN,...,4.0,NaN,45881.0,"[6, 2]",NaN,NaN,['JP'],1,Crunchyroll,"[Drama, Animation]"


In [39]:
df.shape

(1596, 21)

The title of the show is Kite Liberator. I'll research online to find the runtime of the episodes.

After researching this item, I found out this title is actually a movie with a runtime of 58 minutes. I'll change these values accordingly

In [40]:
# replacing values for 'Kite Liberator'
df.loc[1234, ['type']] = 'movie'
df.loc[1234, ['runtime']] = 58
df.loc[1234, ['seasons']] = 0

### Creating dummy columns for each genre

Currently the genres of each title are stored in titles. I want to create dummy columns for each of the unique genres so if a title has the genre in their list the value will be a 1 in that column or else it will be a 0. 

I will use the .get_dummies() method to create the columns and then concat the dataframe with the dummy columns into a new dataframe called `mvp_df` which stands for 'minimum viable product dataframe'.

In [84]:
mvp_df = pd.concat([df, df['genre_ids_fill'].str.join('|').str.get_dummies()], axis=1)

In [85]:
mvp_df.shape

(1596, 41)

### Getting rid of unnecessary columns

The `mvp_df` has extra columns that I don't need for the recommender system. The columns that I need are as follows: `jw_entity_id`, `title`, `release_year`, `type`, `runtime`, `seasons`, and the `genre` columns.

In [86]:
# looking at mvp_df
mvp_df.head(1)

,jw_entity_id,id,title,poster,description,release_year,type,imdb_popularity,tmdb_popularity,imdb_score,...,Made in Europe,Music & Musical,Mystery & Thriller,Other,Reality TV,Romance,Science-Fiction,Sport,War & Military,Western
0,ts28221,28221,Robotech,/poster/260247952/{profile},Robotech is an 85-episode adaptation of three ...,1985.0,show,4459.0,26.408,8.5,...,0,0,0,0,0,0,1,0,0,0


In [87]:
mvp_df.columns

Index(['jw_entity_id', 'id', 'title', 'poster', 'description', 'release_year',
       'type', 'imdb_popularity', 'tmdb_popularity', 'imdb_score',
       'imdb_votes', 'tmdb_score', 'imdb_id', 'tmdb_id', 'genre_ids',
       'age_certification', 'runtime', 'production_countries', 'seasons',
       'streaming_app', 'genre_ids_fill', 'Action & Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'History',
       'Horror', 'Kids & Family', 'Made in Europe', 'Music & Musical',
       'Mystery & Thriller', 'Other', 'Reality TV', 'Romance',
       'Science-Fiction', 'Sport', 'War & Military', 'Western'],
      dtype='object')

In [88]:
# columns to drop
cols_to_drop = [
    'id', 'poster', 'description', 'imdb_popularity', 
    'tmdb_popularity', 'imdb_score', 'imdb_votes', 
    'tmdb_score', 'imdb_id', 'tmdb_id', 'genre_ids',
    'age_certification', 'production_countries',
    'streaming_app', 'genre_ids_fill',
]

# drop columns from mvp_df
mvp_df.drop(columns=cols_to_drop, inplace=True)

Next I have to use .get_dummies() on the `type` column and then drop the `type` column.

In [89]:
# get dummies on the type column
mvp_df = pd.concat([mvp_df,
           pd.get_dummies(mvp_df['type'])],axis=1)

# drop type column
mvp_df.drop(columns=['type'], inplace=True)

Take a look at the cleaned `mvp_df`

In [91]:
mvp_df.head()

,jw_entity_id,title,release_year,runtime,seasons,Action & Adventure,Animation,Comedy,Crime,Documentary,...,Mystery & Thriller,Other,Reality TV,Romance,Science-Fiction,Sport,War & Military,Western,movie,show
0,ts28221,Robotech,1985.0,24.0,3,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,ts199,Black Lagoon,2006.0,25.0,3,1,1,1,1,0,...,1,0,0,0,0,0,0,0,0,1
2,ts25674,Serial Experiments Lain,1998.0,25.0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
3,ts20388,Psycho-Pass,2012.0,27.0,4,1,1,0,1,0,...,1,0,0,0,1,0,1,1,0,1
4,ts27309,Paranoia Agent,2004.0,24.0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1


# First Recommender System

I have to set up two new tables.
1. A look up table that will look up the jw_entity_id and have the corresponding title.
2. The model dataframe that will have the jw_entity_id as the index and drop the title column. This table will be used for the cosine similarity.

In [136]:
# creating look up table
lookup_table = mvp_df[['jw_entity_id','title']].set_index('jw_entity_id')

# model dataframe
model_df = mvp_df.drop(columns=['title']).set_index('jw_entity_id')

In [137]:
model_df

,release_year,runtime,seasons,Action & Adventure,Animation,Comedy,Crime,Documentary,Drama,Fantasy,...,Mystery & Thriller,Other,Reality TV,Romance,Science-Fiction,Sport,War & Military,Western,movie,show
jw_entity_id,,,,,,,,,,,,,,,,,,,,,
ts28221,1985.0,24.0,3,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
ts199,2006.0,25.0,3,1,1,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
ts25674,1998.0,25.0,1,0,1,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
ts20388,2012.0,27.0,4,1,1,0,1,0,0,0,...,1,0,0,0,1,0,1,1,0,1
ts27309,2004.0,24.0,1,0,1,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ts249933,2020.0,4.0,1,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
ts77748,2018.0,23.0,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
ts134409,2015.0,3.0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [138]:
# get the index for a title to test
title_index = lookup_table.index[lookup_table['title'] == 'Attack on Titan']

# get the row of the title_index from model_df
title_array = np.array(model_df.loc[title_index])

# reshape it so it can be passed to cosine_sim function
title_array = title_array.reshape(1,-1)

Now to create cosine similarity matrix using the model_df and the title_array.

In [139]:
title_array

array([[2.013e+03, 2.400e+01, 4.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00]])

In [140]:
# cosine similarity matrix
cosine_matrix = cosine_similarity(model_df, title_array)

# create a dataframe from the cosine_matrix
cosine_df = pd.DataFrame(data=cosine_matrix, index=model_df.index)

In [141]:
# top 10 results of the cosine_df
results = cosine_df.index.values[:11]
results

array(['ts28221', 'ts199', 'ts25674', 'ts20388', 'ts27309', 'ts28010',
       'ts12787', 'ts22056', 'ts22113', 'ts19528', 'ts22595'],
      dtype=object)

These are the index numbers for the title. I'll look up these values in the lookup_table

In [142]:
# look up values in look up table
lookup_table.loc[results]

,title
jw_entity_id,
ts28221,Robotech
ts199,Black Lagoon
ts25674,Serial Experiments Lain
ts20388,Psycho-Pass
ts27309,Paranoia Agent
ts28010,High School DxD
ts12787,Code Geass: Lelouch of the Rebellion
ts22056,TRIGUN
ts22113,Initial D


In [143]:
lookup_table.iloc[:11]

,title
jw_entity_id,
ts28221,Robotech
ts199,Black Lagoon
ts25674,Serial Experiments Lain
ts20388,Psycho-Pass
ts27309,Paranoia Agent
ts28010,High School DxD
ts12787,Code Geass: Lelouch of the Rebellion
ts22056,TRIGUN
ts22113,Initial D


In [144]:
# scaling model_df columns
model_df
cols_to_scale = ['release_year', 'runtime', 'seasons']

# instantiate standard scaler
ss = StandardScaler()
scaled_cols = ss.fit_transform(model_df[cols_to_scale])
scaled = pd.DataFrame(scaled_cols, index=model_df.index, columns=cols_to_scale)

# update columns in model_df
for column in cols_to_scale:
    model_df[column] = scaled[column]

In [145]:
model_df['release_year'] = scaled['release_year']

In [346]:
# get the index for a title to test
title_index = lookup_table.index[lookup_table['title'] == "One Piece"][0]

# get the row of the title_index from model_df
title_array = np.array(model_df.loc[title_index])

# reshape it so it can be passed to cosine_sim function
title_array = title_array.reshape(1,-1)

Now to create cosine similarity matrix using the model_df and the title_array.

In [347]:
title_array

array([[-1.57623995, -0.18108784,  9.57565233,  1.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

In [348]:
# cosine similarity matrix
cosine_matrix = cosine_similarity(model_df, title_array)

# create a dataframe from the cosine_matrix
cosine_df = pd.DataFrame(data=cosine_matrix, index=model_df.index)

In [349]:
# top 10 results of the cosine_df
results = cosine_df.sort_values(0, ascending=False).index.values[:11]
results

array(['ts20339', 'ts22069', 'ts10145', 'ts15151', 'ts32450', 'ts20740',
       'ts29406', 'ts94758', 'ts14234', 'ts29076', 'ts20548'],
      dtype=object)

These are the index numbers for the title. I'll look up these values in the lookup_table

In [350]:
# look up values in look up table
lookup_table.loc[results]

,title
jw_entity_id,
ts20339,One Piece
ts22069,Naruto Shippūden
ts10145,Case Closed
ts15151,Bleach
ts32450,The Prince of Tennis
ts20740,Dragon Ball Z
ts29406,Gintama
ts94758,The Journey Home
ts14234,Cardfight!! Vanguard


In [313]:
lookup_table.iloc[:11]

,title
jw_entity_id,
ts28221,Robotech
ts199,Black Lagoon
ts25674,Serial Experiments Lain
ts20388,Psycho-Pass
ts27309,Paranoia Agent
ts28010,High School DxD
ts12787,Code Geass: Lelouch of the Rebellion
ts22056,TRIGUN
ts22113,Initial D
